# Machine Learning Assessment

## 1. Initialisation

The initialisation modules import and parse the calls and weather data, saved in `init.py`.

In `calls_parser`, the call catabase is parsed. In particular, the dates and times are converted to the standard form with time zone.

There are events at the edge of standard and daylight times, and are ambiguous, since in the database the time zone is not indicated. I decide to drop these events.

In [1]:
%%writefile init.py

import os
import pandas as pd
from time import process_time as timer

def calls_parser(fname='./data/calls.csv') :
    print('Loading raw Seattle 911 calls database from ' + fname)
    tim = timer()
    calls_df = pd.read_csv(fname)
    print('Raw Seattle 911 calls database loaded in ' + str(timer() - tim) + ' s')

    calls_df['Datetime'] = pd.to_datetime(calls_df['Datetime'],\
                                          format="%m/%d/%Y %I:%M:%S %p"\
                                         ).dt.tz_localize(tz='US/Pacific',\
                                                          ambiguous='NaT')

    calls_df.set_index('Datetime', inplace=True)
    calls_df.index.set_names('datetime', inplace=True)
    calls_df.sort_index(inplace=True)
    
    calls_df.dropna(inplace=True) # see section 1 in main.ipynb

    return calls_df


Writing init.py


In `init_calls`, the parsed calls database is saved. I use `parquet` format due to [performance](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#performance-considerations) and compatibility considerations.

In [2]:
%%writefile -a init.py

def init_calls() :
    calls_df_pt = './tmp/calls_df.parquet'
    
    if not os.path.exists('./tmp') :
        os.system('mkdir tmp')
    if os.path.isfile(calls_df_pt) :
        print('Loading parsed Seattle 911 calls database from ' + str(calls_df_pt))
        tim = timer()
        calls_df = pd.read_parquet(calls_df_pt) # fast and compat output
        print('Parsed Seattle 911 calls database loaded in ' + str(timer() - tim) + ' s')
    else :
        calls_pt = './data/calls.csv'
    
        if not os.path.exists(calls_pt) :
            print('Downloading missing raw Seattle 911 calls database to ' + calls_pt)
            os.system('cat get_calls.sh | sh')
    
        calls_df = calls_parser(calls_pt)

        print('Saving parsed Seattle 911 calls database to ' + calls_df_pt)
        tim = timer()
        calls_df.to_parquet(calls_df_pt) # fast and compat input
        print('Parsed Seattle 911 calls database saved in ' + str(timer() - tim) + ' s')
    
    return calls_df

Appending to init.py


In `weather_parser` and `init_weather`, the weather data is parsed and saved.

There are also duplicates, i.e. more than one items within the same hour. I decide to drop them.

In [3]:
%%writefile -a init.py

def weather_parser(fname='./data/Seattle Weatherdata 2002 to 2020.csv') :
    print('Loading raw Seattle weather database from ' + fname)
    tim = timer()
    wtr_df = pd.read_csv(fname)
    print('Raw Seattle weather database loaded in ' + str(timer() - tim) + ' s')

    wtr_df['datetime'] = pd.to_datetime(wtr_df['dt'], unit='s').\
                            dt.tz_localize(tz='UTC').\
                            dt.tz_convert('US/Pacific')
    wtr_df.set_index('datetime', inplace=True)
    
    return(wtr_df)


def init_weather() :
    
    wtr_df_pt = './tmp/wtr_df.parquet'
    
    if not os.path.exists('./tmp') :
        os.system('mkdir tmp')
    if os.path.isfile(wtr_df_pt) :
        print('Loading parsed Seattle weather database from ' + str(wtr_df_pt))
        tim = timer()
        wtr_df = pd.read_parquet(wtr_df_pt)
        print('Parsed Seattle weather database loaded in ' + str(timer() - tim) + ' s')
    else :
        wtr_df = weather_parser(fname='./data/Seattle Weatherdata 2002 to 2020.csv')

        print('Saving parsed Seattle weather database to ' + wtr_df_pt)
        tim = timer()
        wtr_df.to_parquet(wtr_df_pt)
        print('Parsed Seattle weather database saved in ' + str(timer() - tim) + ' s')

    wtr_df = wtr_df[~wtr_df.index.duplicated()] # see main.ipynb

    return(wtr_df)


Appending to init.py


In `feature_parser`, I choose a subset of all weather data as features. The criteria are unfortunately somewhat subjective. Time, day, calendar week and year are also included, because I expect systematic influences by these factors.

In `y_parser`, I cound event numbers from the parsed database.

In `init`, I combine the feature and target databases and save it.

In [4]:
%%writefile -a init.py

def feature_parser(wtr_df) :
    x = wtr_df[['temp', #'temp_min', 'temp_max',
                'pressure', 'humidity', 'wind_speed', 'wind_deg', 'weather_id']]
    x.index.set_names('datetime', inplace=True)

    x_tim = x.index.isocalendar()
    x_tim['hour'] = x.index.hour
    x = x_tim.join(x)

    return x


def y_parser(calls_df) :
    y = calls_df['Incident Number'].resample('H').count().to_frame('incident_count')
    y.index.set_names('datetime', inplace=True)
    
    return y


def init() :
    
    xy_df_pt = './tmp/xy_df.parquet'
    
    if not os.path.exists('./tmp') :
        os.system('mkdir tmp')
    if os.path.isfile(xy_df_pt) :
        print('Loading ML database from ' + str(xy_df_pt))
        tim = timer()
        xy_df = pd.read_parquet(xy_df_pt)
        print('ML database loaded in ' + str(timer() - tim) + ' s')
    else :
        
        tim = timer()
        calls_df = init_calls()
        wtr_df = init_weather()
        
        x_raw = feature_parser(wtr_df)

        y_raw = y_parser(calls_df)
        xy_df = y_raw.join(x_raw).dropna()

        xy_df.drop_duplicates(inplace=True)

        print('Saving ML database to ' + xy_df_pt)
        tim = timer()
        xy_df.to_parquet(xy_df_pt) #, key='x', mode='w'
        print('ML database saved in ' + str(timer() - tim) + ' s')

    return xy_df.iloc[:, 1:], xy_df.iloc[:, 0]


Appending to init.py


## 2. Preprocessing

Here I define the splitters and the preprocessing pipeline in `prep.py`.

In `fwd_splitter`, I use `TimeSeriesSplit` to get a series of back-testing training-testing pairs. Unfortunately this will not be used, since the best fit from e.g. 2003 - 2008 will not be quite useful for predicting events in 2021.

In [5]:
%%writefile prep.py

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
import sklearn.preprocessing as skl_prep
from sklearn.model_selection import TimeSeriesSplit

def fwd_splitter(n_spi=2, tr_week=261, te_week=52) :
    return TimeSeriesSplit(n_splits=n_spi, max_train_size=tr_week*7*24, test_size=te_week*7*24)


Writing prep.py


Here in `nai_splitter` I just naively split the last six years into five years (training) and one year (testing).

In [6]:
%%writefile -a prep.py

def nai_splitter() :
    e1_ind = -365*24
    tr_ind = (-365*5-1)*24+e1_ind
    e2_ind = (-365*5-1)*24+2*e1_ind
    return np.arange(tr_ind, e1_ind), np.arange(e1_ind, 0), np.arange(e2_ind, tr_ind)


Appending to prep.py


In `prep_ppl` I only use the `StandardScaler`, since I do not know what else to do.

In [7]:
%%writefile -a prep.py

def prep_ppl() :
    return(Pipeline([
        ('std_scl', skl_prep.StandardScaler())
    ]))


Appending to prep.py


## 3. Modelling

Here I define the estimator in `model.py`.

Since I do not know too much about machine learning, I just follow the suggestion and choose `GradientBoostingRegressor`.

Following [a suggestion](https://towardsdatascience.com/how-to-generate-prediction-intervals-with-scikit-learn-and-python-ab3899f992ed), I use Huber loss function with different `alpha` to make a prediction interval.

In [8]:
%%writefile model.py

from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
#from sklearn.ensemble import HistGradientBoostingRegressor

import prep

def model_ppl(rseed=0) :
    est_l = Pipeline([
            ('preprocessor', prep.prep_ppl()),
            ('regressor', GradientBoostingRegressor(
                loss='huber',
                alpha=.1,
                random_state=rseed))])
    est_m = Pipeline([
            ('preprocessor', prep.prep_ppl()),
            ('regressor', GradientBoostingRegressor(
                loss='ls',
                random_state=rseed))])
    est_u = Pipeline([
            ('preprocessor', prep.prep_ppl()),
            ('regressor', GradientBoostingRegressor(
                loss='huber',
                alpha=.9,
                random_state=rseed))])

    return est_l, est_m, est_u

Writing model.py


## 4. Training

The training pipeline is contained in `train.py`.

In `train`, I use `GridSearchCV` to decide a better set of parameters. However I do not understand much about them, so I only play with the `n_estimators`.

By the way, the Chinese data scientists also use 'alchemy practice' to indicate training. But here I do not use this term.

In [9]:
%%writefile train.py

import os
from sklearn.model_selection import GridSearchCV
from time import process_time as timer
import joblib

import init, prep, model

def train_cv(est, p_grid, x, y, fname) :
    print('Cross validating by Grid Search')
    reg = GridSearchCV(estimator = est, param_grid=p_grid, n_jobs=-1, verbose=2)
    reg.fit(x, y)
    print(reg.best_params_)
    joblib.dump(reg, fname)
    print('Regressor dumped to' + fname)
    
    return reg

def train(redo = False) :

    if not os.path.exists('./tmp') :
        os.system('mkdir tmp')

    train_l_pt, train_m_pt, train_u_pt = './tmp/reg_l.pkl', './tmp/reg_m.pkl', './tmp/reg_u.pkl'
    if not redo and os.path.isfile(train_l_pt) :
        reg_l = joblib.load(train_l_pt)
        reg_m = joblib.load(train_m_pt)
        reg_u = joblib.load(train_u_pt)
        print('Regressor loaded from ' + train_l_pt + ', ' + train_m_pt + ', '+ train_u_pt)
    else:
        x, y = init.init()
        tr_ind, e1_ind, e2_ind = prep.nai_splitter()
    
        p_grid = {
            # 'regressor__learning_rate': [.1,], # .1
            'regressor__n_estimators': [x + 100 for x in range(5)], # 100
            # 'regressor__subsample': [1.], # 1.
            # 'regressor__max_depth': [3], # 3
            # 'regressor__max_features': ['auto'],
            # 'regressor__tol': [1e-4],
        }

        tim = timer()
        
        est_l, est_m, est_u = model.model_ppl()
        
        reg_l = train_cv(est_l, p_grid, x.iloc[tr_ind], y.iloc[tr_ind], train_l_pt)
        reg_m = train_cv(est_m, p_grid, x.iloc[tr_ind], y.iloc[tr_ind], train_m_pt)
        reg_u = train_cv(est_u, p_grid, x.iloc[tr_ind], y.iloc[tr_ind], train_u_pt)

    return reg_l, reg_m, reg_u


Writing train.py


## 5. Main

In `main.py`, there is only the `main` function. If no input is given, it runs a demo by using a data generator `tester.data_gen`.

In [10]:
%%writefile main.py

import sys, getopt, os
import numpy as np
import pandas as pd

import init, train, tester

def usage() :
    print('usage: python ' + sys.argv[0] + ' -w weather.csv [-c calls.csv] {--week, --month}')

def main() :
    try :
        opts, args = getopt.getopt(sys.argv[1:], 'hw:c:WM', ['help', 'weather=', 'calls=', 'week', 'month'])
    except getopt.GetoptError as err :
        print(err)
        usage()
        sys.exit(2)

    fwtr = ''
    fcal = ''

    if not opts:
        usage()
        sys.exit()

    for o, a in opts :
        if o in ('-h', '--help') :
            usage()
            sys.exit()
        elif o in ('-w', '--weather') :
            fwtr = a
        elif o in ('-c', '--calls') :
            fcal = a
        elif o in ('-W', '--week'):
            fwtr = './tmp/test_wtr.csv'
            fcal = './tmp/test_cal.csv'
            print('Work in back-testing mode (week) with ' + fwtr + ' and ' + fcal)
            tester.data_gen(fwtr, fcal, dend='2020-01-07')
        elif o in ('-M', '--month'):
            fwtr = './tmp/test_wtr.csv'
            fcal = './tmp/test_cal.csv'
            print('Work in back-testing mode (month) with ' + fwtr + ' and ' + fcal)
            tester.data_gen(fwtr, fcal, dend='2020-01-31')

    wtr_new = init.weather_parser(fwtr)
    wtr_new = wtr_new[~wtr_new.index.duplicated()]
    x_new = init.feature_parser(wtr_new)

    reg_l, reg_m, reg_u = train.train()

    if fcal : # real data given
        calls_new = init.calls_parser(fcal)
        y_new = init.y_parser(calls_new)
        tester.printer(x_new, y_new['incident_count'], reg_l, reg_m, reg_u)
    else : # no real data
        y_pel, y_pem, y_peu = reg_l.predict(x_new), reg_m.predict(x_new), reg_u.predict(x_new)
        print('Middle prediction: ' + str(int(y_pem.sum())))
        print('Prediction interval: [' + str(int(y_pel.sum())) + ', ' + str(int(y_peu.sum())) + ']')


if __name__ == '__main__':
    main()


Writing main.py
